# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902008


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:32,  1.13s/it]

Rendering models:  10%|▉         | 3/31 [00:03<00:36,  1.30s/it]

Rendering models:  16%|█▌        | 5/31 [00:09<00:43,  1.67s/it]

Rendering models:  19%|█▉        | 6/31 [00:09<00:35,  1.40s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:22<00:01,  2.36it/s]

Rendering models:  94%|█████████▎| 29/31 [00:23<00:00,  2.02it/s]

Rendering models:  97%|█████████▋| 30/31 [00:24<00:00,  1.74it/s]

Rendering models: 100%|██████████| 31/31 [00:25<00:00,  1.32it/s]

equidad1                              0.000088
JingWei                               0.000668
bobhail                               0.000075
JessieDawn                            0.001761
welshgirlie                           0.009582
kayleebug2017                         0.000200
readsandwiches                        0.000200
thomaskayll                           0.001999
marius_aga                            0.002051
Astronomy4U                           0.000609
not-logged-in-6c3c85931d83a1164a0d    0.000928
not-logged-in-d0bc4156ef5274edc132    0.000102
theetick                              0.003605
sn346808                              0.036995
sn345012                              0.049657
puic                                  0.000156
not-logged-in-2972998ec62ca7e53d73    0.000057
KevinKosa                             0.000130
ranyel.a.e.pasimio                    0.002196
not-logged-in-031d1298fa56e96a6058    0.000174
not-logged-in-ab8497cedb2c38e126d1    0.000114
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())